In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.1


In [2]:
!python -V

Python 3.10.11


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/anaconda/envs/mlops4/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/anaconda/envs/mlops4/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
year = 2023
month = 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
y_pred.std()

np.float64(6.247488852238703)

In [13]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [14]:
df['y_pred'] = y_pred

In [17]:
output_file = f'yellow_tripdata_{year:04d}-{month:02d}_predictions.parquet'
df_result = df[['ride_id', 'y_pred']]

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)



In [18]:
!ls -lah

total 66M
drwxr-xr-x 3 azureuser azureuser 4.0K Apr  9 13:01 .
drwxr-xr-x 7 azureuser azureuser 4.0K Apr  9 12:51 ..
-rw-r--r-- 1 azureuser azureuser 6.8K Apr  9 12:56 homework_4.ipynb
-rw-r--r-- 1 azureuser azureuser  17K Apr  9 12:48 model.bin
-rw-r--r-- 1 azureuser azureuser 4.9K Apr  9 13:02 starter.ipynb
drwxr-xr-x 2 azureuser azureuser 4.0K Apr  9 12:47 web-service
-rw-r--r-- 1 azureuser azureuser  66M Apr  9 13:01 yellow_tripdata_2023-03_predictions.parquet
